# Default prediction for Lending Club data
## Raw Data

Authors : Iker Aguirre, Carlos Serrano

Date : 04/12/2020

The purpose of this work is to create a model in order to predict if a future client of Lending Club will default or pay the full loan.
We will follow the following steps:

1. Loading the data.


2. Preliminary exploratory analysis.


3. Transform the target variable.


4. Feature selection.


5. Drop highly correlated variables.


6. Apply the models


7. Compare the models.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt

# 1. Loading de Data

We currently have access to four different datasets. In order to work with all of them at once, we will use the concat() function from pandas, so we can concatenate them into a single dataset.



In [2]:
data_raw1 = pd.read_csv('Loan_training_set_1_4.csv',
                      skiprows = 1)
data_raw2 = pd.read_csv('Loan_training_set_2_4.csv',
                      skiprows = 1)
data_raw3 = pd.read_csv('Loan_training_set_3_4.csv',
                      skiprows = 1)
data_raw4 = pd.read_csv('Loan_training_set_4_4.csv',
                      skiprows = 1)

/Users/ikeraguirreotaegui/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/ikeraguirreotaegui/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/ikeraguirreotaegui/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0,19,129,130,131,134,135,136,139) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/ikeraguirreotaegui/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:

In [3]:
data = pd.concat([data_raw1, data_raw2, data_raw3, data_raw4])

# 2. Preliminary exploratory analysis

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 884884 entries, 0 to 421096
Columns: 151 entries, id to settlement_term
dtypes: float64(111), object(40)
memory usage: 1.0+ GB


In [5]:
data.head()

id  member_id  loan_amnt  funded_amnt  funded_amnt_inv        term  \
0  112435993        NaN     2300.0       2300.0           2300.0   36 months   
1  112290210        NaN    16000.0      16000.0          16000.0   60 months   
2  112436985        NaN     6025.0       6025.0           6025.0   36 months   
3  112439006        NaN    20400.0      20400.0          20400.0   36 months   
4  112438929        NaN    13000.0      13000.0          13000.0   36 months   

  int_rate  installment grade sub_grade  ... hardship_payoff_balance_amount  \
0   12.62%        77.08     C        C1  ...                            NaN   
1   12.62%       360.95     C        C1  ...                            NaN   
2   15.05%       209.01     C        C4  ...                            NaN   
3    9.44%       652.91     B        B1  ...                            NaN   
4   11.99%       431.73     B        B5  ...                            NaN   

  hardship_last_payment_amount disbursement_method  debt_settlement_flag  \
0                          NaN                Cash                     N   
1                          NaN                Cash                     N   
2                          NaN                Cash                     N   
3                          NaN                Cash                     N   
4                          NaN                Cash                     N   

  debt_settlement_flag_date settlement_status settlement_date  \
0                       NaN               NaN             NaN   
1                       NaN               NaN             NaN   
2                       NaN               NaN             NaN   
3                       NaN               NaN             NaN   
4                       NaN               NaN             NaN   

  settlement_amount settlement_percentage settlement_term  
0               NaN                   NaN             NaN  
1               NaN                   NaN             NaN  
2               NaN                   NaN             NaN  
3               NaN                   NaN             NaN  
4               NaN                   NaN             NaN  

[5 rows x 151 columns]

In [6]:
data.describe()

member_id      loan_amnt    funded_amnt  funded_amnt_inv  \
count        0.0  884876.000000  884876.000000    884876.000000   
mean         NaN   14975.072016   14975.072016     14969.930469   
std          NaN    8804.519267    8804.519267      8801.522079   
min          NaN    1000.000000    1000.000000       900.000000   
25%          NaN    8000.000000    8000.000000      8000.000000   
50%          NaN   13000.000000   13000.000000     13000.000000   
75%          NaN   20000.000000   20000.000000     20000.000000   
max          NaN   40000.000000   40000.000000     40000.000000   

         installment    annual_inc            dti    delinq_2yrs  \
count  884876.000000  8.848760e+05  884615.000000  884876.000000   
mean      440.549628  7.710242e+04      18.903262       0.348848   
std       254.920865  1.354916e+05      11.722350       0.929979   
min        14.010000  0.000000e+00       0.000000       0.000000   
25%       256.870000  4.600000e+04      12.330000       0.000000   
50%       378.200000  6.500000e+04      18.220000       0.000000   
75%       580.730000  9.200000e+04      24.770000       0.000000   
max      1719.830000  1.100000e+08     999.000000      42.000000   

       fico_range_low  fico_range_high  ...  deferral_term  hardship_amount  \
count   884876.000000    884876.000000  ...         2661.0      2661.000000   
mean       694.698726       698.698854  ...            3.0       133.134930   
std         30.936483        30.937106  ...            0.0       113.474583   
min        660.000000       664.000000  ...            3.0         0.640000   
25%        670.000000       674.000000  ...            3.0        46.410000   
50%        685.000000       689.000000  ...            3.0       102.810000   
75%        710.000000       714.000000  ...            3.0       187.880000   
max        845.000000       850.000000  ...            3.0       893.050000   

       hardship_length  hardship_dpd  \
count           2661.0   2661.000000   
mean               3.0     11.635851   
std                0.0     10.335066   
min                3.0      0.000000   
25%                3.0      0.000000   
50%                3.0     12.000000   
75%                3.0     21.000000   
max                3.0     32.000000   

       orig_projected_additional_accrued_interest  \
count                                 2269.000000   
mean                                   396.396148   
std                                    339.570837   
min                                      1.920000   
25%                                    139.170000   
50%                                    304.740000   
75%                                    558.870000   
max                                   2679.150000   

       hardship_payoff_balance_amount  hardship_last_payment_amount  \
count                     2661.000000                   2661.000000   
mean                     10337.284833                    184.650631   
std                       7206.873974                    191.519744   
min                         55.730000                      0.010000   
25%                       4606.530000                     30.560000   
50%                       8882.150000                    124.770000   
75%                      14467.400000                    291.010000   
max                      40306.410000                   1247.480000   

       settlement_amount  settlement_percentage  settlement_term  
count        8520.000000            8520.000000      8520.000000  
mean         5252.542545              48.307514        10.570423  
std          3738.505953               8.882677         8.513096  
min            82.960000               0.550000         0.000000  
25%          2367.000000              45.000000         1.000000  
50%          4429.550000              45.175000        12.000000  
75%          7176.242500              50.000000        18.000000  
max         30000.000000             521.350000        65.00000

In [7]:
list(data)

['id',
 'member_id',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'term',
 'int_rate',
 'installment',
 'grade',
 'sub_grade',
 'emp_title',
 'emp_length',
 'home_ownership',
 'annual_inc',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'url',
 'desc',
 'purpose',
 'title',
 'zip_code',
 'addr_state',
 'dti',
 'delinq_2yrs',
 'earliest_cr_line',
 'fico_range_low',
 'fico_range_high',
 'inq_last_6mths',
 'mths_since_last_delinq',
 'mths_since_last_record',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'initial_list_status',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_d',
 'last_pymnt_amnt',
 'next_pymnt_d',
 'last_credit_pull_d',
 'last_fico_range_high',
 'last_fico_range_low',
 'collections_12_mths_ex_med',
 'mths_since_last_major_derog',
 'policy_code',
 'application_type',
 'annual_inc_joint',
 '

In [4]:
variables = pd.read_excel('LCDataDictionary.xlsx').dropna()
variables.style.set_properties(subset=['Description'], **{'width' :'850px'})

In [9]:
data.to_csv('data.csv')